# Outline: Representation and Selection of Descriptors
1. Description of typical descriptors and libraries for handling biomolecules
	a. Molecular fingerprints (e.g., ECFP, MACCS keys) 
	b. Learned representations (e.g., learned embeddings from molecular strings {SMILES, SELFIES, SMARTS patterns for structural fragments}, 2D graphs).
	c. Atomic-centered descriptors
		i. SOAP
	d. Coulomb matrices (where to put, exactly?)
	e. Physicochemical properties (e.g., molecular weight, logP, perhaps just go over the ones in QM7)

2. Typical data preprocessing steps
	a. Handling data on different scales (normalization and scaling techniques, standardization {e.g., z-score normalization}, min-max scaling, robust scaling 

3. Feature selection
	a.Illustrate with 2D representation, 3D representation, and MD trajectories (3 examples of increasing complexity).

# Description of typical descriptors and libraries for handling biomolecules 
Molecules are complex and can be represented by strings (e.g., SMILES), 2D graphs, and conformers. There are a few libraries for working with molecule "objects" in Python, such as [RDKit](), [OpenEye](), and [](). In this notebook we will use RDKit.

To import the necessary libraries for this tutorial, run the following cell:

In [1]:
from rdkit import Chem       # description
from rdkit.Chem import Draw  # used for drawing molecules

ModuleNotFoundError: No module named 'rdkit'

## Molecular fingerprints